In [0]:
cd drive/My Drive/Dataset

/content/drive/My Drive/Dataset


In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

import matplotlib.pyplot as plt

In [107]:
# (Read Data)
data = pd.read_csv('train.csv')

# 100% (Randomly Smaple data, ratio is 100%)
data = data.sample(frac = 1)

# (Show Data)
data.head()

,id,title,author,text,label
11256,11256,Magnus Carlsen Overwhelms Challenger in Tiebre...,John Leland,"It took nearly three weeks, 12 closely fought ...",0
13044,13044,’Ban Cars to Stop Terror’ Says Sweden’s Best-S...,Virginia Hale,Cars and other vehicles “have turned into dead...,0
8717,8717,Secret Emails From Hillary Clinton Reveal: The...,Walid Shoebat,Email \nReading through Hillary and her ‘gang’...,1
8267,8267,SpaceX Rocket Explodes at Launchpad in Cape Ca...,"Kenneth Chang, Mike Isaac and Matt Richtel",A spectacular explosion of a SpaceX rocket on ...,0
20780,20780,Florida Woman Charged in Death of Infant in ‘C...,Christine Hauser,"Early on Oct. 6, Erin was awakened by the so...",0


In [108]:

# (Create a new column called Length that record every Headline length)
data['Length'] = [len(headline) for headline in data['title'].fillna('')]
data.head()

,id,title,author,text,label,Length
11256,11256,Magnus Carlsen Overwhelms Challenger in Tiebre...,John Leland,"It took nearly three weeks, 12 closely fought ...",0,104
13044,13044,’Ban Cars to Stop Terror’ Says Sweden’s Best-S...,Virginia Hale,Cars and other vehicles “have turned into dead...,0,85
8717,8717,Secret Emails From Hillary Clinton Reveal: The...,Walid Shoebat,Email \nReading through Hillary and her ‘gang’...,1,218
8267,8267,SpaceX Rocket Explodes at Launchpad in Cape Ca...,"Kenneth Chang, Mike Isaac and Matt Richtel",A spectacular explosion of a SpaceX rocket on ...,0,74
20780,20780,Florida Woman Charged in Death of Infant in ‘C...,Christine Hauser,"Early on Oct. 6, Erin was awakened by the so...",0,83


In [109]:
detail = data['Length'].describe()
print(detail)

count    20800.000000
mean        72.325577
std         25.812558
min          0.000000
25%         59.000000
50%         75.000000
75%         87.000000
max        456.000000
Name: Length, dtype: float64


In [110]:
# Get all fake news
fliter = data['label'] == 1
pos = data[fliter]
print('', len(pos))

# Get all true news
fliter = data['label'] == 0
neg = data[fliter]
print('', len(neg))

the_mean = min(len(pos), len(neg))
print("==============================================")

# 抓取所有 Label為1的 資料，並重新洗牌 (Random Shuffle)
p_data = pos.sample(n = the_mean)
print('', len(p_data))

# 抓取所有 Label為0的 資料，並重新洗牌 (Random Shuffle)
n_data = neg.sample(n = the_mean)
print('', len(n_data))

 10413
 10387
 10387
 10387


In [111]:
test_split = 0.2
train_split = 1 - test_split


# (20% for Testing Data, others 80% for Training Data)
p_train_data = p_data.sample(frac = train_split)
p_test_data = p_data.drop(p_train_data.index)

n_train_data = n_data.sample(frac = train_split)
n_test_data = n_data.drop(n_train_data.index)


# (Combined fake news and true news)
train_data = pd.concat([p_train_data, n_train_data])
test_data = pd.concat([p_test_data, n_test_data])

# (Random Shuffle)
train_data = train_data.sample(frac = 1)
test_data = test_data.sample(frac = 1)

x_train_data = train_data['title'].fillna('')
y_train_data = train_data['label']
x_test_data = test_data['title'].fillna('')
y_test_data = test_data['label']

print('Train Data Feature：', len(x_train_data))
print('Train DataLabel：', len(y_train_data))
print('Test Data Feature：', len(x_test_data))
print('Test Data Label：', len(y_test_data))


Train Data Feature： 16620
Train DataLabel： 16620
Test Data Feature： 4154
Test Data Label： 4154


In [0]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [0]:
# (Create a token dictionary)
token_num = 4000 

# (Get a fix length, we chose the mean of the Headline length)
data_length = int(detail['mean'])

# (Word Embeding output vector dimension)
output_length = 32 

dropout = 0.5
lstm_dim = 256


In [0]:
token = Tokenizer(num_words = token_num, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
token.fit_on_texts(x_train_data)

In [0]:
x_train_seq = token.texts_to_sequences(x_train_data)
x_test_seq = token.texts_to_sequences(x_test_data)


In [0]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = data_length)
x_test = sequence.pad_sequences(x_test_seq, maxlen = data_length)

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional, TimeDistributed
from keras.callbacks import EarlyStopping

In [123]:
model = Sequential()
model.add(Embedding(output_dim = output_length, input_dim = token_num, input_length = data_length))
model.add(Dropout(dropout))

# using BLSTM (this will be better than LSTM, Avg acc is around 0.94
model.add(Bidirectional(LSTM(lstm_dim), merge_mode = 'sum'))
model.add(Dropout(dropout))

# using LSTM, Avg acc is around 0.93
#model.add(LSTM(lstm_dim))
#model.add(Dropout(dropout))

model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(dropout))

model.add(Dense(units = 1, activation = 'sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 72, 32)            128000    
_________________________________________________________________
dropout_44 (Dropout)         (None, 72, 32)            0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 256)               591872    
_________________________________________________________________
dropout_45 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_46 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 257       
Total para

In [0]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:

es = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 2)

In [125]:
train_history = model.fit(x = x_train,
                         y = y_train_data,
                         validation_split = 0.2,
                         epochs = 5,
                         batch_size = 512,
                         verbose = 1,
                         )


Train on 13296 samples, validate on 3324 samples
Epoch 1/5
13296/13296 [==============================] - 15s 1ms/step - loss: 0.6279 - acc: 0.6858 - val_loss: 0.4973 - val_acc: 0.8201
Epoch 2/5
13296/13296 [==============================] - 6s 485us/step - loss: 0.3288 - acc: 0.8791 - val_loss: 0.2393 - val_acc: 0.8995
Epoch 3/5
13296/13296 [==============================] - 6s 487us/step - loss: 0.1752 - acc: 0.9343 - val_loss: 0.1835 - val_acc: 0.9302
Epoch 4/5
13296/13296 [==============================] - 6s 487us/step - loss: 0.1202 - acc: 0.9552 - val_loss: 0.1711 - val_acc: 0.9389
Epoch 5/5
13296/13296 [==============================] - 6s 481us/step - loss: 0.0949 - acc: 0.9645 - val_loss: 0.1694 - val_acc: 0.9377


In [126]:
train_history = model.fit(x = x_train,
                         y = y_train_data,
                         validation_split = 0.3,
                         epochs = 2,
                         batch_size = 512,
                         verbose = 1,
                         )

Train on 11634 samples, validate on 4986 samples
Epoch 1/2
11634/11634 [==============================] - 6s 546us/step - loss: 0.0777 - acc: 0.9716 - val_loss: 0.1347 - val_acc: 0.9535
Epoch 2/2
11634/11634 [==============================] - 6s 519us/step - loss: 0.0631 - acc: 0.9780 - val_loss: 0.1470 - val_acc: 0.9513


In [127]:
train_history = model.fit(x = x_train,
                         y = y_train_data,
                         validation_split = 0.4,
                         epochs = 1,
                         batch_size = 512,
                         verbose = 1,
                         )

Train on 9972 samples, validate on 6648 samples
Epoch 1/1
9972/9972 [==============================] - 6s 588us/step - loss: 0.0545 - acc: 0.9803 - val_loss: 0.1242 - val_acc: 0.9604
